In [1]:
import itertools
import random
import numpy as np

In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import combinations
from collections import defaultdict
import time
from numba import njit

#@njit(parallel=True)
def opt_n_m_anonymity_adj(adj_matrix):
    n_m_anonymity_dict = defaultdict(list)
    
    num_nodes = adj_matrix.shape[0]

    for node in range(num_nodes):
        # Step 1: Determine ego state (neighbors and edges)
        neighbors = np.nonzero(adj_matrix[node])[0]
        num_neighbors = len(neighbors)
        
        # Use Numpy operations for efficiency
        neighborhood_edges = int(np.count_nonzero(adj_matrix[neighbors][:, neighbors])/2)

        ego_state = (num_neighbors + 1, neighborhood_edges+num_neighbors)

        # Step 2: Count unique equivalence classes
        n_m_anonymity_dict[ego_state].append(node)

    number_unique = sum(1 for equivalence_class in n_m_anonymity_dict.values() if len(equivalence_class) == 1)

    return n_m_anonymity_dict, number_unique

# Function to perform mutation with 1% constraint
@njit(parallel=True)
def mutate(individual, allowance=0.01, mutation_rate = 0.05):
    mutation_mask = np.random.rand(len(individual)) < mutation_rate
    individual[mutation_mask] = 1 - individual[mutation_mask]
    
    current_num_edges_to_set_one = sum(individual)
    remaining_num_edges_to_set_one = max(1, int(allowance * len(individual))) - current_num_edges_to_set_one

    # if remaining_num_edges_to_set_one > 0:
    #     edges_to_set_one = np.where(individual == 0)[0]
    #     selected_edges = np.random.choice(edges_to_set_one, remaining_num_edges_to_set_one, replace=False)
    #     individual[selected_edges] = 1
    # else:
    # if remaining_num_edges_to_set_one < 0:
    #     # If we've exceeded the constraint, mutate some bits back to 0
    #     edges_to_reset_zero = np.where(individual == 1)[0]
    #     selected_edges_to_reset = np.random.choice(edges_to_reset_zero, -remaining_num_edges_to_set_one, replace=False)
    #     individual[selected_edges_to_reset] = 0

    return individual

@njit(parallel=True)
def crossover(parent1, parent2, n_points=2, allowance=0.01, mutation_rate=0.05):
    if n_points == 111:
        # Initialize child arrays
        child1 = np.zeros_like(parent1)
        child2 = np.zeros_like(parent2)

        # Perform uniform crossover
        for i in range(len(parent1)):
            # Flip a coin for each gene to decide the inheritance
            if np.random.rand() < 0.5:
                child1[i] = parent1[i]
                child2[i] = parent2[i]
            else:
                child1[i] = parent2[i]
                child2[i] = parent1[i]

        # Apply mutation to children to enforce 1% constraint
        child1 = mutate(child1, allowance, mutation_rate)
        child2 = mutate(child2, allowance, mutation_rate)

    else:
        # Generate n crossover points
        crossover_points = np.sort(np.random.choice(len(parent1), n_points, replace=False))

        # Initialize child arrays
        child1 = np.zeros_like(parent1)
        child2 = np.zeros_like(parent2)

        # Perform crossover
        for i in range(n_points + 1):
            start = 0 if i == 0 else crossover_points[i - 1] + 1
            end = len(parent1) if i == n_points else crossover_points[i]

            # Alternate parents for each segment
            if i % 2 == 0:
                child1[start:end] = parent1[start:end]
                child2[start:end] = parent2[start:end]
            else:
                child1[start:end] = parent2[start:end]
                child2[start:end] = parent1[start:end]

        # Apply mutation to children to enforce 1% constraint
        child1 = mutate(child1, allowance, mutation_rate)
        child2 = mutate(child2, allowance, mutation_rate)

    return child1, child2

# Function to calculate n-m-anonymity
def calculate_n_m_anonymity_sorted(graph, sorted_edges, deleted_edges):
    # Create a copy of the graph to avoid modifying the original graph
    modified_graph = graph.copy()

    # Delete edges based on the binary string
    for edge, delete_flag in zip(sorted_edges, deleted_edges):
        if delete_flag:
            modified_graph.remove_edge(*edge)

    # Calculate n-m-anonymity
    modified_graph_adj = nx.to_numpy_array(modified_graph).astype(int)
    _, fitness = opt_n_m_anonymity_adj(modified_graph_adj)

    return fitness

def roulette_wheel_selection(fitness_values, num_parents=100):
    # Calculate the inverted fitness values (subtract from the maximum fitness)
    inverted_fitness = np.nan_to_num((max(fitness_values)+1) - fitness_values)
    
    # Calculate the total inverted fitness
    total_inverted_fitness = np.sum(inverted_fitness)

    # Check if total_inverted_fitness is zero to avoid division by zero
    if total_inverted_fitness == 0:
        # Handle the case where all fitness values are the same or NaN
        return np.random.choice(len(fitness_values), num_parents)
    
    else:
        # Calculate selection probabilities
        selection_probabilities = inverted_fitness / total_inverted_fitness

        # Use np.random.choice for roulette wheel selection
        parents_indices = np.random.choice(len(fitness_values), num_parents, p=selection_probabilities)

        return parents_indices

def fitness_calculator(popn, uniqueness, allowance):
    fitness = []
    penalty = []
    
    for i, indiv in enumerate(popn):
        current_num_edges_to_set_one = sum(indiv)
        remaining_num_edges_to_set_one = current_num_edges_to_set_one -  max(1, int(allowance * len(indiv))) 

        if remaining_num_edges_to_set_one > 0:
            penalty.append(remaining_num_edges_to_set_one)
        else:
            penalty.append(0)

        fitness.append(uniqueness[i] + penalty[i])

    return fitness


def tournament_selection(fitness_values, tournament_size=5, num_parents=100):
    num_individuals = len(fitness_values)
    parents_indices = []

    for _ in range(num_parents):
        # Randomly select individuals for the tournament
        tournament_indices = np.random.choice(num_individuals, tournament_size, replace=False)
        
        # Find the index of the individual with the highest fitness in the tournament
        winner_index = tournament_indices[np.argmin(fitness_bvalues[tournament_indices])]
        
        # Add the winner's index to the list of selected parents
        parents_indices.append(winner_index)

    return parents_indices

In [3]:
popn_init_options = [0.0025, 0.005, 0.02]
p_selection_options = [1,2]
n_points_options = [10,25,100,111]
m_rate_options = [0.0001, 111, 1111]
e_selection_options = [3] 


configs_setups = [50, 25, 13, 5, 2]
number_runs = [10, 20, 40, 80, 1000]

all_combinations = list(itertools.product(popn_init_options, p_selection_options, n_points_options, m_rate_options, e_selection_options))
random.shuffle(all_combinations)

selected_configs = all_combinations[:50]

In [4]:
rd_number = 0
config_number = 0

In [5]:
number_runs[rd_number], rd_number, config_number, 0
sorting = "BC"
popn_init = selected_configs[config_number][0]
p_selection = selected_configs[config_number][1]
n_points = selected_configs[config_number][2]
m_rate = selected_configs[config_number][3] 
e_selection = selected_configs[config_number][4]
number_runs = number_runs[0]
iteration = 0

In [6]:
mediumG = nx.read_edgelist(r"./../../CA-GrQc.txt")
mediumG.remove_edges_from(nx.selfloop_edges(mediumG))

#sorting by closeness centrality
if sorting == "BC":
    # Step 1: Compute betweenness centrality for each node
    betweenness_centrality = nx.betweenness_centrality(mediumG)

    # Step 2: Associate each edge with the average betweenness centrality of its nodes

    # edge_centrality = {(u, v): (betweenness_centrality[u] + betweenness_centrality[v]) / 2
    #                    for u, v in mediumG.edges()}

    edge_centrality = {(u, v): max(betweenness_centrality[u], betweenness_centrality[v])
                    for u, v in mediumG.edges()}

    # Step 3: Sort edges based on the associated centrality values
    sorted_edges = sorted(edge_centrality.keys(), key=lambda edge: edge_centrality[edge], reverse=True)

In [7]:
adj_G = nx.to_numpy_array(mediumG).astype(int)

num_edges = len(mediumG.edges)
allowance = 0.05
#n_points = 100
population_size = 100
#generations = 1000

if m_rate == 111:
    mutation_rate = 0.0005
    eta = 0.000025
elif m_rate == 1111:
    mutation_rate = 0.0005
    eta = 0.00001
else:
    mutation_rate = 0.0001
    eta = 0

saved_fit = []

In [8]:
if rd_number == 0:
    # Initialize the population with at least 1% of edges set to 1
    population = np.zeros((population_size, num_edges), dtype=int)

    #allowance_num = int(allowance*num_edges)
    for i in range(population_size):
        num_edges_to_set_one = max(1, int(popn_init * num_edges))
        edges_to_set_one = np.random.choice(num_edges, num_edges_to_set_one, replace=False)
        population[i, edges_to_set_one] = 1
else:
    popn_file_path = "./BC/CA-GrQc/rd{}/population_config{}.npy".format(rd_number-1, config_number)
    population = np.load(popn_file_path)

In [20]:
count = 2

In [21]:
if count == 1:
    # Evaluate the fitness of each individual
    #fitness_values = np.array([calculate_n_m_anonymity_sorted(mediumG, sorted_edges, individual) for individual in population])
    
    unique_values = np.array([calculate_n_m_anonymity_sorted(mediumG, sorted_edges,individual) for individual in population])
    fitness_values = np.array(fitness_calculator(population, unique_values, allowance))

print("{},{},{},{}, {}".format(rd_number, config_number, iteration, count, np.min(fitness_values)))
saved_fit.append(np.min(fitness_values))

0,0,0,2, 263


In [11]:
unique_values

array([297, 292, 306, 281, 282, 312, 275, 306, 308, 294, 286, 300, 295,
       314, 286, 269, 302, 284, 297, 294, 303, 290, 302, 291, 299, 291,
       291, 284, 301, 291, 292, 294, 296, 294, 296, 292, 298, 290, 295,
       288, 289, 291, 287, 278, 311, 306, 308, 281, 285, 318, 294, 296,
       298, 292, 284, 290, 284, 294, 301, 296, 287, 297, 300, 285, 301,
       297, 286, 299, 281, 290, 304, 303, 293, 292, 291, 309, 276, 314,
       289, 299, 298, 306, 285, 295, 299, 299, 296, 291, 269, 306, 305,
       304, 286, 290, 286, 284, 307, 279, 296, 291])

In [12]:
fitness_values

array([297, 292, 306, 281, 282, 312, 275, 306, 308, 294, 286, 300, 295,
       314, 286, 269, 302, 284, 297, 294, 303, 290, 302, 291, 299, 291,
       291, 284, 301, 291, 292, 294, 296, 294, 296, 292, 298, 290, 295,
       288, 289, 291, 287, 278, 311, 306, 308, 281, 285, 318, 294, 296,
       298, 292, 284, 290, 284, 294, 301, 296, 287, 297, 300, 285, 301,
       297, 286, 299, 281, 290, 304, 303, 293, 292, 291, 309, 276, 314,
       289, 299, 298, 306, 285, 295, 299, 299, 296, 291, 269, 306, 305,
       304, 286, 290, 286, 284, 307, 279, 296, 291])

In [13]:
if p_selection == 1:
    parents_indices = roulette_wheel_selection(fitness_values, num_parents = 150)
elif p_selection == 2:
    parents_indices = tournament_selection(fitness_values, tournament_size = 5, num_parents = 150)

In [15]:
# Create the next generation through crossover and mutation
children = []

mutation_rate = max(1/num_edges, mutation_rate * (1 - eta * count))

# Create the next generation through crossover and mutation
children = []
for i in range(0, len(parents_indices)-1, 2):  # Ensure that the loop doesn't go out of bounds
    parent1, parent2 = population[parents_indices[i]], population[parents_indices[i + 1]]
    child1, child2 = crossover(parent1, parent2, n_points, allowance, mutation_rate)
    children.extend([child1, child2])

In [16]:
# Evaluate fitness values of children
unique_values_children = np.array([calculate_n_m_anonymity_sorted(mediumG, sorted_edges, child) for child in children])
fitness_values_children = np.array(fitness_calculator(children, unique_values_children, allowance))

In [17]:
parents_children = np.concatenate((population, children))
parents_children_fitness = np.hstack((fitness_values, fitness_values_children))
parents_children_unique = np.hstack((unique_values, unique_values_children))

sorted_parents_children = np.argsort(parents_children_fitness)
parents_children = parents_children[sorted_parents_children]
parents_children_fitness = parents_children_fitness[sorted_parents_children]
parents_children_unique = parents_children_unique[sorted_parents_children]

In [18]:
if e_selection == 1:
    new_parents_indices = roulette_wheel_selection(sorted_parents_children, num_parents = 100)
    population = parents_children[new_parents_indices]
    fitness_values = parents_children_fitness[new_parents_indices]
    unique_values = parents_children_unique[new_parents_indices]
elif e_selection == 2:
    new_parents_indices = tournament_selection(sorted_parents_children, 5, num_parents = 100)
    population = parents_children[new_parents_indices]
    fitness_values = parents_children_fitness[new_parents_indices]
    unique_values = parents_children_unique[new_parents_indices]
else:
    population = parents_children[0:population_size]
    fitness_values = parents_children_fitness[0:population_size]
    unique_values = parents_children_unique[0:population_size]

In [19]:
unique_values

array([263, 265, 269, 269, 269, 273, 273, 273, 274, 274, 275, 276, 276,
       276, 276, 276, 277, 277, 277, 278, 278, 278, 279, 279, 279, 279,
       280, 280, 280, 280, 280, 281, 281, 281, 281, 281, 281, 282, 282,
       282, 282, 282, 282, 283, 283, 283, 283, 283, 283, 283, 283, 284,
       284, 284, 284, 284, 284, 284, 284, 284, 284, 284, 284, 284, 284,
       284, 284, 285, 285, 285, 285, 285, 285, 285, 286, 286, 286, 286,
       286, 286, 286, 286, 286, 287, 287, 287, 287, 288, 288, 288, 288,
       289, 289, 289, 289, 289, 289, 289, 289, 290])

In [6]:
for rd_number in range(len(number_runs)):

    for config_number in range(configs_setups[rd_number]):
        if rd_number == 0:
            print(config_number)
        else:
            print(0, config_number)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 0
0 1
0 2
0 3
0 4
0 0
0 1
